In [1]:
import sys
import numpy as np
import pandas as pd
import random
from tqdm.auto import tqdm
from transformers import pipeline, set_seed
import torch
from rdkit import Chem, RDLogger
lg = RDLogger.logger()
lg.setLevel(RDLogger.CRITICAL)

2022-12-12 13:42:22.338386: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-12 13:42:22.523359: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-12 13:42:23.132207: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-12 13:42:23.132318: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

# Setup tokenization

## Atom-based tokenization & BPE class

In [2]:
import collections
import logging
import os
import re
import codecs
import unicodedata
from typing import List, Optional
from transformers import PreTrainedTokenizer
from SmilesPE.tokenizer import SPE_Tokenizer

def load_vocab(vocab_file):
    """Loads a vocabulary file into a dictionary."""
    vocab = collections.OrderedDict()
    with open(vocab_file, "r", encoding="utf-8") as reader:
        tokens = reader.readlines()
    for index, token in enumerate(tokens):
        token = token.rstrip("\n")
        vocab[token] = index
    return vocab

class Atomwise_Tokenizer(object):
    """Run atom-level SMILES tokenization"""

    def __init__(self):
        """ Constructs a atom-level Tokenizer.
        """
        self.regex_pattern = r"(\[[^\]]+]|Br?|Cl?|N|O|S|P|F|I|b|c|n|o|s|p|\(|\)|\.|=|#|-|\+|\\|\/|:|~|@|\?|>>?|\*|\$|\%[0-9]{2}|[0-9])"
        self.regex = re.compile(self.regex_pattern)

    def tokenize(self, text):
        """ Basic Tokenization of a SMILES.
        """
        tokens = [token for token in self.regex.findall(text)]
        return tokens

class SMILES_SPE_Tokenizer(PreTrainedTokenizer):
    r"""
    Constructs a SMILES tokenizer. Based on SMILES Pair Encoding (https://github.com/XinhaoLi74/SmilesPE).
    This tokenizer inherits from :class:`~transformers.PreTrainedTokenizer` which contains most of the methods. Users
    should refer to the superclass for more information regarding methods.
    Args:
        vocab_file (:obj:`string`):
            File containing the vocabulary.
        spe_file (:obj:`string`):
            File containing the trained SMILES Pair Encoding vocabulary.
        unk_token (:obj:`string`, `optional`, defaults to "[UNK]"):
            The unknown token. A token that is not in the vocabulary cannot be converted to an ID and is set to be this
            token instead.
        sep_token (:obj:`string`, `optional`, defaults to "[SEP]"):
            The separator token, which is used when building a sequence from multiple sequences, e.g. two sequences
            for sequence classification or for a text and a question for question answering.
            It is also used as the last token of a sequence built with special tokens.
        pad_token (:obj:`string`, `optional`, defaults to "[PAD]"):
            The token used for padding, for example when batching sequences of different lengths.
        cls_token (:obj:`string`, `optional`, defaults to "[CLS]"):
            The classifier token which is used when doing sequence classification (classification of the whole
            sequence instead of per-token classification). It is the first token of the sequence when built with
            special tokens.
        mask_token (:obj:`string`, `optional`, defaults to "[MASK]"):
            The token used for masking values. This is the token used when training this model with masked language
            modeling. This is the token which the model will try to predict.
    """

    def __init__(
        self,
        vocab_file,
        spe_file,
        unk_token="[UNK]",
        bos_token="[BOS]",
        eos_token="[EOS]",
        # sep_token="[SEP]",
        pad_token="[PAD]",
        # cls_token="[CLS]",
        # mask_token="[MASK]",
        **kwargs
    ):
        super().__init__(
            unk_token=unk_token,
            bos_token=bos_token,
            eos_token=eos_token,
            pad_token=pad_token,
            # cls_token=cls_token,
            # mask_token=mask_token,
            **kwargs,
        )

        if not os.path.isfile(vocab_file):
            raise ValueError(
                "Can't find a vocabulary file at path '{}'.".format(vocab_file)
            )
        if not os.path.isfile(spe_file):
            raise ValueError(
                "Can't find a SPE vocabulary file at path '{}'.".format(spe_file)
            )
        self.vocab = load_vocab(vocab_file)
        self.spe_vocab = codecs.open(spe_file)
        self.ids_to_tokens = collections.OrderedDict([(ids, tok) for tok, ids in self.vocab.items()])
        self.spe_tokenizer = SPE_Tokenizer(self.spe_vocab)

    @property
    def vocab_size(self):
        return len(self.vocab)

    def get_vocab(self):
        return dict(self.vocab, **self.added_tokens_encoder)

    def _tokenize(self, text):
        return self.spe_tokenizer.tokenize(text).split(' ')

    def _convert_token_to_id(self, token):
        """ Converts a token (str) in an id using the vocab. """
        return self.vocab.get(token, self.vocab.get(self.unk_token))

    def _convert_id_to_token(self, index):
        """Converts an index (integer) in a token (str) using the vocab."""
        return self.ids_to_tokens.get(index, self.unk_token)

    def convert_tokens_to_string(self, tokens):
        """ Converts a sequence of tokens (string) in a single string. """
        out_string = "".join(tokens).replace(" ##", "").strip()
        return out_string

    def build_inputs_with_special_tokens(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None
    ) -> List[int]:
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A BERT sequence has the following format:
        - single sequence: ``[CLS] X [SEP]``
        - pair of sequences: ``[CLS] A [SEP] B [SEP]``
        Args:
            token_ids_0 (:obj:`List[int]`):
                List of IDs to which the special tokens will be added
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.
        Returns:
            :obj:`List[int]`: list of `input IDs <../glossary.html#input-ids>`__ with the appropriate special tokens.
        """
        bos = [self.bos_token_id]
        eos = [self.eos_token_id]
        output = bos + token_ids_0 + eos
        if token_ids_1 is None:
            return output
        return output + [bos] + token_ids_1 + [eos]

    def get_special_tokens_mask(
        self, token_ids_0: List[int], token_ids_1: Optional[List[int]] = None, already_has_special_tokens: bool = False
    ) -> List[int]:
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` method.
        Args:
            token_ids_0 (:obj:`List[int]`):
                List of ids.
            token_ids_1 (:obj:`List[int]`, `optional`, defaults to :obj:`None`):
                Optional second list of IDs for sequence pairs.
            already_has_special_tokens (:obj:`bool`, `optional`, defaults to :obj:`False`):
                Set to True if the token list is already formatted with special tokens for the model
        Returns:
            :obj:`List[int]`: A list of integers in the range [0, 1]: 1 for a special token, 0 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]


    def save_vocabulary(self, vocab_path):
        """
        Save the sentencepiece vocabulary (copy original file) and special tokens file to a directory.
        Args:
            vocab_path (:obj:`str`):
                The directory in which to save the vocabulary.
        Returns:
            :obj:`Tuple(str)`: Paths to the files saved.
        """
        index = 0
        if os.path.isdir(vocab_path):
            vocab_file = os.path.join(vocab_path, VOCAB_FILES_NAMES["vocab_file"])
        else:
            vocab_file = vocab_path
        with open(vocab_file, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.vocab.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(vocab_file)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1
        return (vocab_file,)

tokenizer = SMILES_SPE_Tokenizer(
    vocab_file='./tokenization/SPE/vocab_spe.txt', 
    spe_file= './tokenization/SPE/SPE_ChEMBL.txt'
)

# Setup inference

In [24]:
from transformers import pipeline, TextGenerationPipeline

class MyPipeline(TextGenerationPipeline):
    def postprocess(self, model_outputs):
        output_tokens = np.squeeze(model_outputs['generated_sequence']).numpy()
        output = tokenizer.decode(output_tokens, skip_special_tokens=True)
        return {'generated': output}
        
model_path = './save/irak4-20221212-1339/checkpoint-84/'
config_path = f"{model_path}config.json"
device = 1 # -1 stands for using CPU

generators = []

# Top-K Sampling
# ref: https://huggingface.co/blog/how-to-generate#top-k-sampling
generators.append(
    pipeline(
    'text-generation', 
    model=model_path,
    config=config_path,
    tokenizer=tokenizer,
    device=device,
    max_length=100,
    pad_token_id=0,
    do_sample=True, 
    top_k=50, 
    early_stopping=True,
    pipeline_class=MyPipeline
))

# Top-P Sampling
# ref: https://huggingface.co/blog/how-to-generate#top-p-nucleus-sampling
generators.append(
    pipeline(
    'text-generation', 
    model=model_path,
    config=config_path,
    tokenizer=tokenizer,
    device=device,
    max_length=100,
    pad_token_id=0,
    do_sample=True, 
    top_p=.92, 
    top_k=0, 
    early_stopping=True,
    pipeline_class=MyPipeline
))

# Contrastive Search
# https://huggingface.co/blog/introducing-csearch#contrastive_objective
generators.append(
    pipeline(
    'text-generation', 
    model=model_path,
    config=config_path,
    tokenizer=tokenizer,
    device=device,
    max_length=100,
    pad_token_id=0,
    do_sample=True, 
    top_k=4, 
    penalty_alpha=0.6,
    early_stopping=True,
    pipeline_class=MyPipeline
))

# Data generator

In [25]:
gen_num = 30_000
prompt = '[BOS]'

def data(gen_num):
    count = gen_num
    while count:
        count-=1
        yield prompt

In [10]:
%%time
# Non-parallel generation version
results = []
for generator in tqdm(generators, position=0, desc='generator#', leave=False):
    results.append([
        cmpd['generated'] 
        for cmpd in tqdm(
            generator(data(gen_num)), 
            position=1, desc='cmpd#', leave=False, total=gen_num
        )
    ])
    # print(out['generated'])

generator#:   0%|          | 0/3 [00:00<?, ?it/s]

cmpd#:   0%|          | 0/30000 [00:00<?, ?it/s]


KeyboardInterrupt



In [ ]:
%%time
# Parallel generation version
# https://joblib.readthedocs.io/en/latest/parallel.html
from joblib import Parallel, delayed

def batch_generation(generator, data, gen_num):
    return [
        cmpd['generated'] 
        for cmpd in tqdm(generator(data(gen_num)), total=gen_num)
    ]

results = Parallel(n_jobs=128)(
    delayed(batch_generation)
    (generator, data, gen_num) 
    for generator in tqdm(generators)
)

  0%|          | 0/3 [00:00<?, ?it/s]

2022-12-13 16:12:37.497063: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-13 16:12:38.793585: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-12-13 16:12:38.793829: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2022-12-13 16:12:38.804347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-12-13 16:12:45.565748: E tensorflow/stream_executor/cuda/c

In [23]:
df = pd.DataFrame(results, index=None).T
df.rename({0: 'top-k', 1: 'top-p', 2: 'cons'}, axis=1, inplace=True)
df.to_csv('./generation/irak4_spe/checkpoint-70-30_000_gen_sample.csv', index=False)